## Using a tool

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [2]:
tool.name

'wikipedia'

In [3]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [4]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [5]:
tool.return_direct

False

In [6]:
tool.run({"query": "Founder of Wikipedia"})

'Page: Founders of Wikipedia\nSummary: There are two co-founders of Wikipedia:\n\nLarry Sanger (born 1968), American internet project developer\nJimmy Wales (born 1966), American-British internet entrepreneur'

## Custom tools

Can be defined in 3 ways: 

- decorator 
- Subclassing the class BaseModel 

### 1 Decorators

In [7]:
from langchain.tools import tool


@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search("What's the best application framework for LLMs?")

/tmp/ipykernel_41523/3717988365.py:9: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search("What's the best application framework for LLMs?")


'LangChain'

In [8]:
print(search.name)
print(search.description)

search
Look up things online.


In [9]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search_basemodel(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search_basemodel.name)
print(search_basemodel.description) 

search_basemodel("How do we implement LLM apps?")

search-tool
Look up things online.


'LangChain'

In [10]:
try:
    search-tool("How do we implement LLM apps?")
except TypeError as e:
    print(f'TypeError: {e}')

TypeError: unsupported operand type(s) for -: 'StructuredTool' and 'function'


As a conclusion from the above there is a point which I do not understand why this defined that way:
The tool name is search-tool but I need to use the function "search_basemodell" to use it. I find this strange.

Also using the search-tool name does not work. So this is not an alternative way of calling the function in a different way, It is only a possibility to give a different name, Which however also cannot be used anywhere. 


### Subclassing BaseTool

In [11]:
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class CustomSearchTool(BaseTool):
    name: str = "custom_search"
    description: str = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

search = CustomSearchTool()
search("What's the most popular tool for writing LLM apps?")

'LangChain'

### StrucuredTool dataclass

In [12]:
from langchain.tools import StructuredTool

def search_function(query: str):
    return "LangChain Bug"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
)
search("Which framework has hundreds of integrations to use with LLMs?")

'LangChain Bug'

In [14]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
) 

calculator.run(dict(a=1_000_000_000, b=23))

23000000000

## Error handling

In [17]:
from langchain_core.tools import ToolException

def search_tool(s: str):
    if 7 == 9:
        return "This is a mathematicall wonder!"
    else:
        raise ToolException("The search tool is not available.")

search = StructuredTool.from_function(
    func=search_tool,
    name="Search_tool",
    description="A bad tool",
    handle_tool_error=True,
)
search("Search the internet and compress everything into a paragraph!")

'The search tool is not available.'